<a href="https://colab.research.google.com/github/mayank-soni/text_summary/blob/transformer_train/label_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 32.4 MB/s 
     |████████████████████████████████| 7.6 MB 54.6 MB/s 
     |████████████████████████████████| 182 kB 66.8 MB/s 


In [3]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 23.6 MB/s 
     |████████████████████████████████| 212 kB 68.8 MB/s 
     |████████████████████████████████| 115 kB 66.3 MB/s 
     |████████████████████████████████| 127 kB 67.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [8]:
from transformers import pipeline
from tqdm import tqdm

In [6]:
classifier = pipeline("zero-shot-classification", device = 0, model = 'valhalla/distilbart-mnli-12-1')

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/890M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [7]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
candidate_labels = ['about sports', 'not about sports']

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [22]:
dataset_subset = dataset['test'].shard(num_shards=100, index=0)

In [ ]:
# for out in tqdm(classifier(KeyDataset(dataset_subset, "article"), candidate_labels)):
#     labels.append({'label': out['labels'][0], 'score': out['scores'][0]})

In [29]:
output = classifier(dataset_subset['article'], candidate_labels)

In [30]:
subset_indices = []

In [36]:
for i, article in enumerate(output):
  if article['labels'][0] == 'about sports' and article['scores'][0]>0.6:
    subset_indices.append(i)

In [37]:
subset_indices

[16, 14, 15, 16, 22, 30, 33, 40, 43]

In [41]:
a = dataset_subset.select(subset_indices)

In [42]:
a.save_to_disk('sports_test.hf')

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

In [39]:
for case in output:
  if case['labels'][0] == 'about sports' and case['scores'][0]>0.6:
    print(case)

{'sequence': "Four years after becoming the youngest first-class cricketer in county history, Yorkshire's Barney Gibson has retired from the sport. The Leeds-born wicketkeeper entered the record books in 2011 when he lined up against Durham University just 27 days after his 15th birthday. But that match proved to be his only appearance at senior level and he never again progressed from the second XI. Ben Gibson, pictured at the age of 15, has decided to retire from cricket just four years after his debut . The 19-year-old said it was a 'difficult decision' to retire from cricket at such a young age . In his last game for the second string he did not bat or keep wicket, instead sending down 3.3 overs for 29 runs. 'This was a difficult decision to make,' the 19-year-old said. 'I would like to thank the players and staff at Yorkshire for their support. I have been involved with the club since I was 11 and I feel that now is the right time for me to look at a career change. 'The support fr

In [80]:
for case in output:
  if (case['labels'][0] == 'about sports' and case['scores'][0]<0.7):
    print(case)

{'sequence': '(CNN)A freshly fallen tree in the roadway was Jason Warnock\'s first clue. Warnock was driving through a canyon in Lewiston, Idaho, on Wednesday when he saw the tree, then looked up to see an SUV dangling over the edge of a cliff. The only thing holding the GMC Yukon and its terrified driver from a 30-foot drop was a crumpled chain-link fence, still clinging to the earth above Bryden Canyon Road. "I seen that guy hanging there and he was trying to beat the window out or the door open and I was like \'Oh man,\' \'cause only like five links were hanging there," Warnock told KXLY, a CNN affiliate. "I was like, I gotta do something and no one was doing anything." What Warnock did next, captured in a dramatic photo by Lewiston Tribune photographer Barry Kough, made headlines around the world. Warnock dashed from his car and scrambled up a hill to the Yukon and its driver, 23-year-old Matthew Sitko, who appeared to be in shock. "I got up there and I was like, \'Are you alright 

In [68]:

output[0].keys()

dict_keys(['sequence', 'labels', 'scores'])